In [1]:
import random
import numpy as np
from qiskit import*
from qiskit.circuit.library import UnitaryGate

In [2]:
I2 = np.matrix([[1,0],[0,1]])
H  = np.matrix([[1,1],[1,-1]])* (1/np.sqrt(2))
S  = np.matrix([[1,0],[0,1j]])
X  = np.matrix([[0,1],[1,0]])
Y  = np.matrix([[0,-1j],[1j,0]])
Z  = np.matrix([[1,0],[0,-1]])

r"""

Clifford group is defined as the quotient group C_{n}/U(1) which counts elements in C_{n} that differs
only by an overall global phase factor as the same element. In that case, any element of C_{n}/U(1)
can be written as a product a*b where a \in A and b \in B.

"""
A = [I2,S,H,H@S,S@H,H@S@H]
B = [I2,X,Y,Z]

single_qubit_Clifford_group = []
for a in A:
    for b in B:
        single_qubit_Clifford_group.append(a@b)

In [3]:
# Hamiltonians and Lindbladians
b       = 0.1
gamma_1 = 0.2
gamma_2 = 0.3

Hamiltonian = (-b/2)*X
L_1 = np.sqrt(gamma_1)*np.matrix([[0,1],[1,0]])
L_2 = np.sqrt(gamma_2)*Z

# steady state density matrix parameters
Delta = 2*b**2+gamma_1**2+4*gamma_1*gamma_2
yss   = (2*b*gamma_1)/Delta
zss   = (gamma_1*(gamma_1+4*gamma_2))/Delta

# steady state density matrix
rho_ss = (1/2)*(I2+yss*Y+zss*Z)

# angles for steady state preparation
theta_x = np.arccos(zss)
theta_y = np.arccos(-yss/(4*np.sqrt(1+zss**2)))

In [4]:
# function to be determined
def f(rho,X_p,X_q,X_r):
    return np.tr(X_p@rho@X_q@rho@X_r)

In [5]:
def classical_shadow():
    # setting up the quantum circuit
    density_matrix_qubit = QuantumRegister(1, r"\rho")
    ancilla_qubit = QuantumRegister(1, "ancilla")
    cbit = ClassicalRegister(1,"cbit")
    qc = QuantumCircuit(density_matrix_qubit,ancilla_qubit,cbit)
    # initializing the ancilla qubit at |0>.
    qc.initialize([1,0],1)
    qc.barrier()
    qc.rx(theta_x,0)
    qc.cry(theta_y,0,1)

    # random Clifford rotation
    U = random.choice(single_qubit_Clifford_group)
    random_Clifford_gate = UnitaryGate(U.tolist())
    qc.append(random_Clifford_gate,[0])
    
    # measuring the qubit after Clifford rotation
    qc.measure(0,0)

    # Use Aer's qasm_simulator
    backend_sim = Aer.get_backend('qasm_simulator')

    # Execute the circuit on the qasm simulator.
    number_of_shots = 1
    job_sim = backend_sim.run(transpile(qc, backend_sim), shots=number_of_shots)

    # Grab the results from the job.
    result_sim = job_sim.result()
    counts = result_sim.get_counts(qc)        
        
    try:
        counts["0"] == 0
        b_ket = np.matrix([[1,0]]).conj().T
    except KeyError:
        b_ket = np.matrix([[0,1]]).conj().T
        
    n = 1
    rho_hat = (2**n+1)*(U.conj().T)@b_ket@(b_ket.conj().T)@U - np.identity(2**n)
    return rho_hat,qc

In [49]:
rho_1 = classical_shadow()[0]
rho_2 = classical_shadow()[0]
(34/0.1**2)*8*np.sqrt(np.var([np.trace(rho_1@rho_2),np.trace(Y@rho_1@rho_2),np.trace(Z@rho_1@rho_2),np.trace(X@rho_1@X@rho_2),np.trace(Y@rho_1@Y@rho_2),
      np.trace(Z@rho_1@Z@rho_2),np.trace(X@rho_1@Y@rho_2),np.trace(Y@rho_1@Z@rho_2)]))

49911.722070070864

In [7]:
2*np.log(2*8/0.01)

14.755517816455745

In [8]:
N = 100
K = 14

rho_hat_lst = []
for i in range(N*K):
    rho_hat = classical_shadow()[0]
    rho_hat_lst.append(rho_hat)

In [9]:
O_i = X
o_k_lst = []
for k in range(K):
    o_k_i = 0
    # construction of o^{k}_{i}(N,1)
    for j in range(N*(k-1)+1,N*k+1,1):
        for l in range(N*(k-1)+1,N*k+1,1):
            if j != l:
                rho_hat_j = rho_hat_lst[j]
                rho_hat_l = rho_hat_lst[l]
                o_k_i += np.trace(O_i@rho_hat_j@rho_hat_l)
    o_k_lst.append(o_k_i/(N*(N-1)))
np.median(o_k_lst)

(-8.691139839237337e-18-2.6017347647781444e-18j)